In [1]:
import wbgapi as wb
import pandas as pd
import numpy as np
from IPython.display import display
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

### EXTRACTING, RESHAPING AND PROCESSING DATA FROM WORLD BANK

In [3]:
class ExtractWorldBankData:

    datasets = {
        'GE.EST' : 'Government Effectiveness: Estimate',
        'PV.EST' : 'Political Stability and Absence of Violence/Terrorism: Estimate',
        'CC.EST' : 'Control of Corruption: Estimate',
        'IQ.CPA.TRAN.XQ' : 'CPIA transparency, accountability, and corruption in the public sector rating (1=low to 6=high)',
        'RL.EST' : 'Rule of Law: Estimate',
        'SI.POV.NAHC' : 'Poverty headcount ratio at national poverty lines (% of population)',
        'SL.UEM.TOTL.NE.ZS' : 'Unemployment, total (% of total labor force) (national estimate)',
        'NY.GDP.PCAP.KD' : 'GDP per capita (constant 2015 US$)',
        'NY.GDP.PCAP.CD' : 'GDP per capita (current US$)',
        'NY.GDP.MKTP.KD.ZG' : 'GDP growth (annual %)',
        'NY.GDP.MKTP.KN' : 'GDP (constant LCU)',
        'FP.CPI.TOTL.ZG' : 'Inflation, consumer prices (annual %)',
        'BN.CAB.XOKA.GD.ZS' : 'Current account balance (% of GDP)',
        'BN.CAB.XOKA.CD' : 'Current account balance (BoP, current US$)',
        'DT.TDS.DECT.EX.ZS' : 'Total debt service (% of exports of goods, services and primary income)',
        'FM.LBL.BMNY.GD.ZS' : 'Broad money (% of GDP)',
        'FM.LBL.BMNY.ZG' : 'Broad money growth (annual %)',
        'PA.NUS.FCRF' : 'Official exchange rate (LCU per US$, period average)',
        'PX.REX.REER' : 'Real effective exchange rate index (2010 = 100)'
    }
    
    def __init__(self):
        self.results = {}
        self.combined_df = pd.DataFrame()

    def fetch_and_process_data(self):
        for data_id, data_name in self.datasets.items():
            print(f'Processing id: {data_id} : {data_name}')
            
            data = wb.data.DataFrame(data_id, numericTimeKeys=True, labels=True).reset_index()
            
            filtered_data = data[data['Country'] == 'Pakistan'].copy()
            
            filtered_data['Variable Name'] = data_name
            
            year_columns = [col for col in filtered_data.columns if str(col).isdigit() and int(col) > 2016]
            columns_to_keep = ['economy', 'Country', 'Variable Name'] + year_columns
            df = filtered_data[columns_to_keep]
            
            self.results[data_name] = df
            
            self.combined_df = pd.concat([self.combined_df, df], ignore_index=True)

    def return_display_data(self, name=None):
        if name:
            if name in self.results:
                display(self.results[name])
            else:
                raise ValueError(f"No data found for dataset name: {name}")
        else:
            return self.combined_df
            # display(self.combined_df)
    
    # def reshaped_return_display_data(self):
    #     df = self.combined_df.drop(columns=['economy', 'Country'])
    #     df_melted = df.melt(id_vars=['Variable Name'], var_name='Year', value_name='Value')
    #     df_pivoted = df_melted.pivot_table(index='Year', columns='Variable Name', values='Value')
    #     return df_pivoted


    def reshaped_return_display_data(self):
        df = self.combined_df.drop(columns=['economy', 'Country'])
        
        # Reshape the dataframe by melting and then pivoting
        df_melted = df.melt(id_vars=['Variable Name'], var_name='Year', value_name='Value')
        df_pivoted = df_melted.pivot_table(index='Year', columns='Variable Name', values='Value')
        
        # Reset the column name for 'Year'
        df_pivoted.reset_index(inplace=True)  # 'Year' becomes a column instead of an index
    
        # Optional: Ensure the 'Year' column is converted to integer if necessary
        df_pivoted['Year'] = df_pivoted['Year'].astype(int)
        
        return df_pivoted
    
    
    def save_excel(self, output_path):
        self.df.to_excel(output_path, index=False)
        return self

# processor = ExtractWorldBankData()
# processor.fetch_and_process_data()
# # processor.display_data('Broad money (% of GDP)')
# # processor.display_data()
# df = processor.reshaped_return_display_data()
# display(df)

In [4]:
# Data from world bank
wrldb = ExtractWorldBankData()
wrldb.fetch_and_process_data()
wrldb_df = wrldb.reshaped_return_display_data()
wrldb_df.to_excel("worldbanvariables.xlsx")
# display(wrldb_df)

Processing id: GE.EST : Government Effectiveness: Estimate
Processing id: PV.EST : Political Stability and Absence of Violence/Terrorism: Estimate
Processing id: CC.EST : Control of Corruption: Estimate
Processing id: IQ.CPA.TRAN.XQ : CPIA transparency, accountability, and corruption in the public sector rating (1=low to 6=high)
Processing id: RL.EST : Rule of Law: Estimate
Processing id: SI.POV.NAHC : Poverty headcount ratio at national poverty lines (% of population)
Processing id: SL.UEM.TOTL.NE.ZS : Unemployment, total (% of total labor force) (national estimate)
Processing id: NY.GDP.PCAP.KD : GDP per capita (constant 2015 US$)
Processing id: NY.GDP.PCAP.CD : GDP per capita (current US$)
Processing id: NY.GDP.MKTP.KD.ZG : GDP growth (annual %)
Processing id: NY.GDP.MKTP.KN : GDP (constant LCU)
Processing id: FP.CPI.TOTL.ZG : Inflation, consumer prices (annual %)
Processing id: BN.CAB.XOKA.GD.ZS : Current account balance (% of GDP)
Processing id: BN.CAB.XOKA.CD : Current account ba

### EXTRACTING, RESHAPING AND PROCESSING DATA FROM STATE BANK

In [6]:
class StateBankDataExtraction:

    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None
        self.processingvariables = None
        self.dashboardvariables = None

    def load_excel(self, sheet_name=None, header=0):
        self.df = pd.read_excel(self.file_path, sheet_name=sheet_name, header=header)
        return self

    def handle_duplicates(self):
        self.df['Duplicate Count'] = self.df.groupby(['Sector', 'Sub-Sector', 'Org Name']).cumcount()
        self.df['Org Name'] = self.df.apply(
            lambda row: f"{row['Org Name'].split(' - ')[0]}{row['Duplicate Count']} - {row['Org Name'].split(' - ')[1]}"
            if row['Duplicate Count'] > 0 else row['Org Name'],
            axis=1)
        self.df = self.df.drop(columns=['Duplicate Count'])
        return self

    def fill_empty_rows(self):
        current_sector = None
        current_subsector = None
        current_org_name = None
        for index, row in self.df.iterrows():  
            if pd.notna(row['Sector']):
                current_sector = row['Sector']
            else:
                self.df.at[index, 'Sector'] = current_sector  
            if pd.notna(row['Sub-Sector']):
                current_subsector = row['Sub-Sector']
            else:
                self.df.at[index, 'Sub-Sector'] = current_subsector 

            if pd.notna(row['Org Name']):
                current_org_name = row['Org Name']
            else:
                self.df.at[index, 'Org Name'] = current_org_name
        return self

    def filter_data(self):
        values_to_remove = [
        ('All Public / Private', 'All Public / Private', '716 - All Public / Private'),
        ('Private', 'Private', '718 - Private'),
        ('Public', 'Public', '717 - Public'),
        ('All Sector', 'All Sector', '703 - All Sector'),
        ('Textile Sector', 'Textile Sector' , '704 - Textile Sector'),
        ('Textile Sector', 'Made-up textile articles', '724 - Made-up textile articles'),
        ('Textile Sector', 'Other textiles n.e.s.', '725 - Other textiles n.e.s.'),
        ('Textile Sector', 'Spinning, Weaving, Finishing of Textiles', '723 - Spinning, Weaving, Finishing of Textiles'),
        ('Sugar', 'Sugar', '726 - Sugar'),
        ('Food', 'Food', '727 - Food'),
        ('Chemicals, Chemical Products and Pharmaceuticals', 'Chemicals, Chemical Products and Pharmaceuticals', '706 - Chemicals, Chemical Products and Pharmaceuticals'),
        ('Manufacturing', 'Manufacturing', '712 - Manufacturing'),
        ('Mineral products', 'Mineral products', '728 - Mineral products'),
        ('Cement', 'Cement', '729 - Cement'),
        ('Motor Vehicles, Trailers & Autoparts', 'Motor Vehicles, Trailers & Autoparts', '714 - Motor Vehicles, Trailers & Autoparts'),
        ('Fuel and Energy Sector', 'Fuel and Energy Sector', '710 - Fuel and Energy Sector'),
        ('Information and Communication Services', 'Information and Communication Services', '711 - Information and Communication Services'),
        ('Coke and Refined Petroleum Products', 'Coke and Refined Petroleum Products', '705 - Coke and Refined Petroleum Products'),
        ('Paper, Paperboard and Products', 'Paper, Paperboard and Products', '709 - Paper, Paperboard and Products'),
        ('Electrical Machinery and Apparatus', 'Electrical Machinery and Apparatus', '707 - Electrical Machinery and Apparatus'),
        ('Other Services Activities', 'Other Services Activities', '715 - Other Services Activities')
        ]        
        condition = self.df.apply(lambda row: (row['Sector'], row['Sub-Sector'], row['Org Name']) in values_to_remove, axis=1)
        self.df = self.df[~condition]  
        return self

    def filter_variables(self):
        self.df['Item Name'] = self.df['Item Name'].str.strip()
        variables_to_keep = [
            '1. Capital work in progress',
            '4. Intangible assets',
            '6. Other non-current assets',
            'Total Assets (A+B) / Equity & Liabilities (C+D+E)',
            'C. Shareholders\' Equity (C1+C2+C3)',
            'D. Non-Current Liabilities (D1+D2+D3+D4+D5)',
            'E. Current Liabilities (E1+E2+E3+E4)',
            'of which: i) Interest / markup payables',
            'ii) Taxes payable',
            '1. Sales',
            '2. Cost of sales',
            '3. Gross profit / (loss) (F1-F2)',
            '4. General, administrative and other expenses',
            'ii) Administrative and other expenses',
            '6. EBIT (F3-F4+F5)',
            '7. Financial expenses',
            '8. Profit / (loss) before taxation (F6-F7)',
            '9. Tax expenses',
            '10. Profit / (loss) after tax (F8-F9)',
            '11. Cash dividends',
            '12. Bonus shares / stock dividends',
            '1. Total capital employed (C+D)',
            '2. Retention in business (F10-F11-F12)',
            '4. Salaries, wages and employee benefits',
            '5. Total fixed liabilities (D1+D3)',
            '6. Contractual liabilities (H5+E2(i))',
            '7. Purchases (F2+Current year B2 - Prev. Year B2)',
            '8. Operating cash flow (F6+H3-F9(i(a))',
            'I. Key Performance Indicators',
            'P5. Return on equity (F10 as % of Avg {Current year(C),previous year (C)}',
            'L2. Quick ratio (B1+B3+B5 to E)',
            'S1. Debt equity ratio [(D+E) to C]',
            'S2. Debt to assets ratio (  D+E as % of Avg. {Current year(A+B),previous year (A+B)})',
            'Profitability Ratios',
            'P1. Net Profit  margin / Net profit to sales (F10 as % of F1)',
            'P2. Asset turnover (F1 to Avg {Current year(A+B),previous year (A+B)}),',
            'P3. Return on Assets  (F10 as a % of Avg {Current year(A+B),previous year (A+B)}',
            'P4. Financial leverage (Avg. {Current year(A+B),previous year (A+B) to Avg. Current year(C),previous year (C))})',
            'P5. Return on equity (F10 as % of Avg {Current year(C),previous year (C)}',
            'P6. Gross profit  margin / Gross profit to sales (F3 as % of F1)',
            'P7. Operating return on assets (F6 as a % of Avg. {Current year(A+B),previous year (A+B)}',
            'P8. Return on capital employed(F7 as a % of Avg {Current year H1, previous year H1}',
            'Liquidity Ratios',
            'L1. Current ratio (B to E)',
            'L2. Quick ratio (B1+B3+B5 to E)',
            'L3. Cash to current liabilities (B1+B5 to E)',
            'Activity Ratios',
            'AC1. Inventory Turnover Ratio (F1 to B2)',
            'AC2. No. of days in inventory (365 to AC1)',
            'AC3. Receivables turnover ratio (F1 to Avg. {Current year(B4),previous year (B4)})',
            'AC4. No. of days in receivables (365 to AC3)',
            'AC5. Payable turnover ratio (H7 to Avg. {Current year(E1(i),previous year(E1 (i)})',
            'AC6. No. of days in payable  (365 to AC5)',
            'AC7. Working capital turnover (F1 to B-E)',
            'AC8. Cash conversion cycle (AC2+AC4-AC6)',
            'Cash Flow Ratios',
            'CF1. Cash flow from operations to sales (G1 to F1)',
            'CF2. Cash return on assets (G1 as % of Avg. {Current year(A+B),previous year (A+B)})',
            'CF3. Cash return on equity (H8 to Avg. {Current year(C),previous year (C)})',
            'CF4. Cash to income (H8 to F10)',
            'CF5. Debt coverage ratio (H8 to (D+E))',
            'Valuation Ratios',
            'V1. Paid up value of shares (Rs.)',
            'V2. Market price per share',
            'V3. Basic earnings per share (Rs./share) ( F10 to No. of shares)',
            'V4. Price earnings ratio (V2 to V3)',
            'V5. Dividend payout ratio (F11 to F10)',
            'V6. Cash dividend per share (Rs./share) (F11 to No. of shares)',
            'V7. Book value per share (Rs./share) (C to No. of shares)',
            'Solvency Ratios',
            'S1. Debt equity ratio [(D+E) to C]',
            'S2. Debt to assets ratio (  D+E as % of Avg. {Current year(A+B),previous year (A+B)})',
            'S3. Debt to capital ratio (D+E to H1)',
            'S4. Interest cover ratio ( F6 to F7(i))'
        ]
        self.df = self.df[self.df['Item Name'].isin(variables_to_keep)]
        return self

    def processing_variable(self):
        self.df['Item Name'] = self.df['Item Name'].str.strip()
        variables_to_keep = [
            '1. Capital work in progress',
            '4. Intangible assets',
            '6. Other non-current assets',
            'Total Assets (A+B) / Equity & Liabilities (C+D+E)',
            'C. Shareholders\' Equity (C1+C2+C3)',
            'D. Non-Current Liabilities (D1+D2+D3+D4+D5)',
            'E. Current Liabilities (E1+E2+E3+E4)',
            'of which: i) Interest / markup payables',
            'ii) Taxes payable',
            '1. Sales',
            '2. Cost of sales',
            '3. Gross profit / (loss) (F1-F2)',
            '4. General, administrative and other expenses',
            'ii) Administrative and other expenses',
            '6. EBIT (F3-F4+F5)',
            '7. Financial expenses',
            '8. Profit / (loss) before taxation (F6-F7)',
            '9. Tax expenses',
            '10. Profit / (loss) after tax (F8-F9)',
            '11. Cash dividends',
            '12. Bonus shares / stock dividends',
            '1. Total capital employed (C+D)',
            '2. Retention in business (F10-F11-F12)',
            '4. Salaries, wages and employee benefits',
            '5. Total fixed liabilities (D1+D3)',
            '6. Contractual liabilities (H5+E2(i))',
            '7. Purchases (F2+Current year B2 - Prev. Year B2)',
            '8. Operating cash flow (F6+H3-F9(i(a))',
            'I. Key Performance Indicators',
            'P5. Return on equity (F10 as % of Avg {Current year(C),previous year (C)}',
            'L2. Quick ratio (B1+B3+B5 to E)',
            'S2. Debt to assets ratio (  D+E as % of Avg. {Current year(A+B),previous year (A+B)})',
            'S1. Debt equity ratio [(D+E) to C]'
        ]
        self.processingvariables = self.df[self.df['Item Name'].isin(variables_to_keep)]
        
        # Reshape the data into the format required by AltmanScore
        df_long = self.processingvariables.melt(
            id_vars=['Sector', 'Sub-Sector', 'Org Name', 'Item Name'], 
            var_name='Year', value_name='Value'
        )
        
        # Pivot the data to get financial metrics in columns
        df_pivot = df_long.pivot_table(
            index=['Sector', 'Sub-Sector', 'Org Name', 'Year'], 
            columns='Item Name', values='Value'
        ).reset_index()
        
        df_pivot.columns.name = None  # Remove the columns name
        self.processingvariables = df_pivot
        self.processingvariables.to_excel('processingvariables.xlsx')
        return self.processingvariables

    def dashboard_variable(self):
        self.df['Item Name'] = self.df['Item Name'].str.strip()
        variables_to_keep = [
            'Profitability Ratios',
            'P1. Net Profit  margin / Net profit to sales (F10 as % of F1)',
            'P2. Asset turnover (F1 to Avg {Current year(A+B),previous year (A+B)}),',
            'P3. Return on Assets  (F10 as a % of Avg {Current year(A+B),previous year (A+B)}',
            'P4. Financial leverage (Avg. {Current year(A+B),previous year (A+B) to Avg. Current year(C),previous year (C))})',
            'P5. Return on equity (F10 as % of Avg {Current year(C),previous year (C)}',
            'P6. Gross profit  margin / Gross profit to sales (F3 as % of F1)',
            'P7. Operating return on assets (F6 as a % of Avg. {Current year(A+B),previous year (A+B)}',
            'P8. Return on capital employed(F7 as a % of Avg {Current year H1, previous year H1}',
            'Liquidity Ratios',
            'L1. Current ratio (B to E)',
            'L2. Quick ratio (B1+B3+B5 to E)',
            'L3. Cash to current liabilities (B1+B5 to E)',
            'Activity Ratios',
            'AC1. Inventory Turnover Ratio (F1 to B2)',
            'AC2. No. of days in inventory (365 to AC1)',
            'AC3. Receivables turnover ratio (F1 to Avg. {Current year(B4),previous year (B4)})',
            'AC4. No. of days in receivables (365 to AC3)',
            'AC5. Payable turnover ratio (H7 to Avg. {Current year(E1(i),previous year(E1 (i)})',
            'AC6. No. of days in payable  (365 to AC5)',
            'AC7. Working capital turnover (F1 to B-E)',
            'AC8. Cash conversion cycle (AC2+AC4-AC6)',
            'Cash Flow Ratios',
            'CF1. Cash flow from operations to sales (G1 to F1)',
            'CF2. Cash return on assets (G1 as % of Avg. {Current year(A+B),previous year (A+B)})',
            'CF3. Cash return on equity (H8 to Avg. {Current year(C),previous year (C)})',
            'CF4. Cash to income (H8 to F10)',
            'CF5. Debt coverage ratio (H8 to (D+E))',
            'Valuation Ratios',
            'V1. Paid up value of shares (Rs.)',
            'V2. Market price per share',
            'V3. Basic earnings per share (Rs./share) ( F10 to No. of shares)',
            'V4. Price earnings ratio (V2 to V3)',
            'V5. Dividend payout ratio (F11 to F10)',
            'V6. Cash dividend per share (Rs./share) (F11 to No. of shares)',
            'V7. Book value per share (Rs./share) (C to No. of shares)',
            'Solvency Ratios',
            'S1. Debt equity ratio [(D+E) to C]',
            'S2. Debt to assets ratio (  D+E as % of Avg. {Current year(A+B),previous year (A+B)})',
            'S3. Debt to capital ratio (D+E to H1)',
            'S4. Interest cover ratio ( F6 to F7(i))'
        ]
        self.dashboardvariables = self.df[self.df['Item Name'].isin(variables_to_keep)]
        
        # Reshape the data into the format required
        df_long = self.dashboardvariables.melt(
            id_vars=['Sector', 'Sub-Sector', 'Org Name', 'Item Name'], 
            var_name='Year', value_name='Value'
        )
        
        df_pivot = df_long.pivot_table(
            index=['Sector', 'Sub-Sector', 'Org Name', 'Year'], 
            columns='Item Name', values='Value'
        ).reset_index()
        
        df_pivot.columns.name = None
        self.dashboardvariables = df_pivot
        return self.dashboardvariables

    # def save_excel(self, output_path):
    #     self.processingvariables.to_excel(output_path, index=False)
    #     return self

    def display_data(self):
        print("Full DataFrame:")
        display(self.df)
        print("\nProcessing Variables DataFrame:")
        display(self.processingvariables)
        print("\nDashboard Variables DataFrame:")
        display(self.dashboardvariables)


In [7]:
#Get data from State Bank
sb = StateBankDataExtraction('FSA(work).xlsx')
sb.load_excel(sheet_name='FSA NF 2017-22', header=3) \
         .handle_duplicates() \
         .fill_empty_rows() \
         .filter_data() \
         .filter_variables()

sb_prc = sb.processing_variable()
sb_prc.to_excel("Processingvariables.xlsx")

sb_das = sb.dashboard_variable()
sb_das.to_excel("Dashboardvariables.xlsx")
sb_df = sb.processing_variable()  # Used in merging futher
display(sb_df)

,Sector,Sub-Sector,Org Name,Year,1. Capital work in progress,1. Sales,1. Total capital employed (C+D),10. Profit / (loss) after tax (F8-F9),11. Cash dividends,12. Bonus shares / stock dividends,...,E. Current Liabilities (E1+E2+E3+E4),I. Key Performance Indicators,L2. Quick ratio (B1+B3+B5 to E),"P5. Return on equity (F10 as % of Avg {Current year(C),previous year (C)}",S1. Debt equity ratio [(D+E) to C],"S2. Debt to assets ratio ( D+E as % of Avg. {Current year(A+B),previous year (A+B)})",Total Assets (A+B) / Equity & Liabilities (C+D+E),ii) Administrative and other expenses,ii) Taxes payable,of which: i) Interest / markup payables
0,Cement,Cement,380001 - Power Cement,2017,235751.0,4480623.0,9005891.0,466793.0,0.00,0.00,...,2381223.0,0.0,1.73865,8.69070,0.35654,0.33952,11387114.0,76366.0,0.0,11214.0
1,Cement,Cement,380001 - Power Cement,2018,14610588.0,4343240.0,21279247.0,319907.0,0.00,0.00,...,3238059.0,0.0,0.51321,3.24889,1.16985,0.73630,24517306.0,202918.0,0.0,249507.0
2,Cement,Cement,380001 - Power Cement,2019,27373414.0,3858455.0,30503208.0,582106.0,0.00,0.00,...,9769846.0,0.0,0.10620,5.08997,2.47973,0.88592,40273054.0,285706.0,0.0,1153972.0
3,Cement,Cement,380001 - Power Cement,2020,23413542.0,4113353.0,25982885.0,-3621629.0,0.00,0.00,...,19518320.0,0.0,0.03983,-36.12195,4.36657,0.86326,45501205.0,458394.0,0.0,1777146.0
4,Cement,Cement,380001 - Power Cement,2021,0.0,14220613.0,29290600.0,358359.0,0.00,0.00,...,16201078.0,0.0,0.03437,3.72833,3.23379,0.76372,45491678.0,285429.0,0.0,1258520.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2250,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340272 - Amtex Ltd.,2022,16638.0,1250878.0,-7373849.0,103489.0,0.00,0.00,...,11035022.0,0.0,0.02443,-1.13159,-1.40264,3.52416,3661173.0,60434.0,0.0,2735144.0
2251,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340793 - INTERLOOP (Pvt) Ltd.,2019,1254704.0,37511062.0,23642522.0,5060551.0,2616592.50,0.00,...,16973454.0,0.0,0.64950,28.89121,1.31881,0.56874,40615976.0,2742842.0,0.0,113942.0
2252,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340793 - INTERLOOP (Pvt) Ltd.,2020,739767.0,36797159.0,26537573.0,1125927.0,1744395.00,0.00,...,18712383.0,0.0,0.40403,6.66604,1.78203,0.67512,45249956.0,2836346.0,0.0,210282.0
2253,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340793 - INTERLOOP (Pvt) Ltd.,2021,2543689.0,54962265.0,32697560.0,6291571.0,2180493.75,261659.25,...,27997279.0,0.0,0.56889,34.21217,1.95862,0.75851,60694839.0,3715659.0,0.0,221674.0


In [8]:
### NEW ALTMAN CALCULATIONS

import numpy as np
import pandas as pd

class AltmanScore:
    
    def __init__(self, df_pivot):
        self.df_pivot = df_pivot

    def calculate_altman_score(self):
        # Assuming df_pivot is already in the correct format
        df_pivot = self.df_pivot.copy()
        
        # Rename the columns to match the Altman Z-Score formula
        df_pivot.rename(columns={
            '1. Capital work in progress': 'WC', 
            'Total Assets (A+B) / Equity & Liabilities (C+D+E)': 'TA', 
            '2. Retention in business (F10-F11-F12)': 'RE', 
            '6. EBIT (F3-F4+F5)': 'EBIT',
            '2. Cost of sales': 'MVE',
            '5. Total fixed liabilities (D1+D3)': 'TL'
        }, inplace=True)

        # Debugging condition
        # Check for zero values in Total Assets and Total Liabilities
        if (df_pivot['TA'] == 0).any():
            print("Warning: Total Assets contains zero values.")
        else:
            print("No need working good")
        
        if (df_pivot['TL'] == 0).any():
            print("Warning: Total Liabilities contains zero values.")
        else:
            print("No need working good")

        print("removingstart")
        df_pivot['WC'] = df_pivot['WC'].replace(0, np.nan)
        df_pivot['TL'] = df_pivot['TL'].replace(0, np.nan)
        df_pivot['TA'] = df_pivot['TA'].replace(0, np.nan)
        df_pivot['MVE'] = df_pivot['MVE'].replace(0, np.nan)
        df_pivot['EBIT'] = df_pivot['EBIT'].replace(0, np.nan)
        df_pivot['RE'] = df_pivot['RE'].replace(0, np.nan)
        print("removingend")
        
        # Debugging condition
        # Check for zero values in Total Assets and Total Liabilities
        if (df_pivot['TA'] == 0).any():
            print("Warning: Total Assets contains zero values.")
        else:
            print("No need working good")
        
        if (df_pivot['TL'] == 0).any():
            print("Warning: Total Liabilities contains zero values.")
        else:
            print("No need working good")
        
        # # Calculate the Altman Z-Score components, handling zeros
        # df_pivot['A'] = df_pivot['WC'] / df_pivot['TA'].replace(0, np.nan)  # Liquidity
        # df_pivot['B'] = df_pivot['RE'] / df_pivot['TA'].replace(0, np.nan)  # Leverage
        # df_pivot['C'] = df_pivot['EBIT'] / df_pivot['TA'].replace(0, np.nan)  # Profitability
        # df_pivot['D'] = df_pivot['MVE'] / df_pivot['TL'].replace(0, np.nan)  # Solvency


        # Forchecking
        df_pivot['A'] = df_pivot['WC'] / df_pivot['TA']
        df_pivot['B'] = df_pivot['RE'] / df_pivot['TA']
        df_pivot['C'] = df_pivot['EBIT'] / df_pivot['TA']
        df_pivot['D'] = df_pivot['MVE'] / df_pivot['TL']
        
        # Replace infinite values with NaN
        df_pivot.replace([np.inf, -np.inf], np.nan, inplace=True)
        
        df_pivot.to_excel("beforealtmanscore.xlsx", index=False)
        
        # Calculate the Altman Z-Score
        df_pivot['Altman Z-Score'] = (3.25 +
                                      6.56 * df_pivot['A'].fillna(0) + 
                                      3.26 * df_pivot['B'].fillna(0) + 
                                      6.72 * df_pivot['C'].fillna(0) + 
                                      1.05 * df_pivot['D'].fillna(0))

        self.df_pivot = df_pivot
        self.df_pivot.to_excel("Withaltmanscore.xlsx", index=False)
        print(df_pivot.columns)  # Check if columns have been renamed properly
        return df_pivot

    def cal_altman_for_each_org_of_each_year(self, save=False, filename='cal_altman_for_each_org_of_each_year.xlsx'):

        pivotted_alt_df = self.df_pivot[['Sector', 'Sub-Sector', 'Org Name', 'Year', 'Altman Z-Score']]
        if save:
            pivotted_alt_df.to_excel(filename, index=False)
            print(f"DataFrame saved to '{filename}'")
        
        return pivotted_alt_df

    def cal_altman_for_each_org_avg(self, save=False, filename='cal_altman_for_each_org_avg.xlsx'):

        filtered_df = self.df_pivot[['Sector', 'Sub-Sector', 'Org Name', 'Altman Z-Score']]
        
        # Filter out 0 and NaN values before calculating the average
        filtered_df = filtered_df[(filtered_df['Altman Z-Score'] != 0) & (filtered_df['Altman Z-Score'].notna())]

        df_avg_z_score = filtered_df.groupby(['Sector', 'Sub-Sector', 'Org Name'])['Altman Z-Score'].mean().reset_index()
        df_avg_z_score.rename(columns={'Altman Z-Score': 'Average Z-Score'}, inplace=True)
        
        if save:
            df_avg_z_score.to_excel(filename, index=False)
            print(f"DataFrame saved to '{filename}'")
        
        return df_avg_z_score

    def cal_altman_for_each_sub_sector(self, save=False, filename='cal_altman_for_each_sub_sector.xlsx'):

        filtered_df = self.df_pivot[['Sector', 'Sub-Sector', 'Altman Z-Score']]
        
        # Filter out 0 and NaN values before calculating the average
        filtered_df = filtered_df[(filtered_df['Altman Z-Score'] != 0) & (filtered_df['Altman Z-Score'].notna())]

        df_avg_z_score_subsector = filtered_df.groupby(['Sector', 'Sub-Sector'])['Altman Z-Score'].mean().reset_index()
        df_avg_z_score_subsector.rename(columns={'Altman Z-Score': 'Average Z-Score'}, inplace=True)
        
        if save:
            df_avg_z_score_subsector.to_excel(filename, index=False)
            print(f"DataFrame saved to '{filename}'")
        
        return df_avg_z_score_subsector

    def cal_altman_for_each_sector(self, save=False, filename='cal_altman_for_each_sector.xlsx'):

        filtered_df = self.df_pivot[['Sector', 'Altman Z-Score']]
        
        # Filter out 0 and NaN values before calculating the average
        filtered_df = filtered_df[(filtered_df['Altman Z-Score'] != 0) & (filtered_df['Altman Z-Score'].notna())]

        df_avg_z_score_sector = filtered_df.groupby(['Sector'])['Altman Z-Score'].mean().reset_index()
        df_avg_z_score_sector.rename(columns={'Altman Z-Score': 'Average Z-Score'}, inplace=True)
        
        if save:
            df_avg_z_score_sector.to_excel(filename, index=False)
            print(f"DataFrame saved to '{filename}'")
        
        return df_avg_z_score_sector


In [9]:
### OLD ALTMAN METHOD

# class AltmanScore:
    
#     def __init__(self, df_pivot):
#         self.df_pivot = df_pivot

#     def calculate_altman_score(self):
#         # Assuming df_pivot is already in the correct format
#         df_pivot = self.df_pivot.copy()
        
#         # Rename the columns to match the Altman Z-Score formula
#         df_pivot.rename(columns={
#             '1. Capital work in progress': 'WC', 
#             'Total Assets (A+B) / Equity & Liabilities (C+D+E)': 'TA', 
#             '2. Retention in business (F10-F11-F12)': 'RE', 
#             '6. EBIT (F3-F4+F5)': 'EBIT',
#             '2. Cost of sales': 'MVE',
#             '5. Total fixed liabilities (D1+D3)': 'TL'
#         }, inplace=True)
        
#         # Calculate the Altman Z-Score components
#         df_pivot['A'] = df_pivot['WC'] / df_pivot['TA']      # Liquidity
#         df_pivot['B'] = df_pivot['RE'] / df_pivot['TA']      # Leverage
#         df_pivot['C'] = df_pivot['EBIT'] / df_pivot['TA']    # Profitability
#         df_pivot['D'] = df_pivot['MVE'] / df_pivot['TL']     # Solvency
        
#         # df_pivot.dropna(subset=['A', 'B', 'C', 'D'], inplace=True)

#         # Calculate the Altman Z-Score
#         df_pivot['Altman Z-Score'] = (3.25 +
#                                       6.56 * df_pivot['A'].fillna(0) + 
#                                       3.26 * df_pivot['B'].fillna(0) + 
#                                       6.72 * df_pivot['C'].fillna(0) + 
#                                       1.05 * df_pivot['D'].fillna(0))

#         self.df_pivot = df_pivot
#         self.df_pivot.to_excel("Withaltmanscore.xlsx")
#         return df_pivot

#     def cal_altman_for_each_org_of_each_year(self, save=False, filename='cal_altman_for_each_org_of_each_year.xlsx'):

#         pivotted_alt_df = self.df_pivot[['Sector', 'Sub-Sector', 'Org Name', 'Year', 'Altman Z-Score']]
#         if save:
#             pivotted_alt_df.to_excel(filename, index=False)
#             print(f"DataFrame saved to '{filename}'")
        
#         return pivotted_alt_df

#     def cal_altman_for_each_org_avg(self, save=False, filename='cal_altman_for_each_org_avg.xlsx'):

#         filtered_df = self.df_pivot[['Sector', 'Sub-Sector', 'Org Name', 'Altman Z-Score']]
        
#         # Filter out 0 and NaN values before calculating the average
#         filtered_df = filtered_df[(filtered_df['Altman Z-Score'] != 0) & (filtered_df['Altman Z-Score'].notna())]

#         df_avg_z_score = filtered_df.groupby(['Sector', 'Sub-Sector', 'Org Name'])['Altman Z-Score'].mean().reset_index()
#         df_avg_z_score.rename(columns={'Altman Z-Score': 'Average Z-Score'}, inplace=True)
        
#         if save:
#             df_avg_z_score.to_excel(filename, index=False)
#             print(f"DataFrame saved to '{filename}'")
        
#         return df_avg_z_score

#     def cal_altman_for_each_sub_sector(self, save=False, filename='cal_altman_for_each_sub_sector.xlsx'):

#         filtered_df = self.df_pivot[['Sector', 'Sub-Sector', 'Altman Z-Score']]
        
#         # Filter out 0 and NaN values before calculating the average
#         filtered_df = filtered_df[(filtered_df['Altman Z-Score'] != 0) & (filtered_df['Altman Z-Score'].notna())]

#         df_avg_z_score_subsector = filtered_df.groupby(['Sector', 'Sub-Sector'])['Altman Z-Score'].mean().reset_index()
#         df_avg_z_score_subsector.rename(columns={'Altman Z-Score': 'Average Z-Score'}, inplace=True)
        
#         if save:
#             df_avg_z_score_subsector.to_excel(filename, index=False)
#             print(f"DataFrame saved to '{filename}'")
        
#         return df_avg_z_score_subsector

#     def cal_altman_for_each_sector(self, save=False, filename='cal_altman_for_each_sector.xlsx'):

#         filtered_df = self.df_pivot[['Sector', 'Altman Z-Score']]
        
#         # Filter out 0 and NaN values before calculating the average
#         filtered_df = filtered_df[(filtered_df['Altman Z-Score'] != 0) & (filtered_df['Altman Z-Score'].notna())]

#         df_avg_z_score_sector = filtered_df.groupby(['Sector'])['Altman Z-Score'].mean().reset_index()
#         df_avg_z_score_sector.rename(columns={'Altman Z-Score': 'Average Z-Score'}, inplace=True)
        
#         if save:
#             df_avg_z_score_sector.to_excel(filename, index=False)
#             print(f"DataFrame saved to '{filename}'")
        
#         return df_avg_z_score_sector


In [10]:
# Get altman score from state bank data
altm = AltmanScore(sb_df)
altm.calculate_altman_score()
altm_sb_df = altm.cal_altman_for_each_org_of_each_year(True)
display(altm_sb_df)

removingstart
removingend
No need working good
No need working good
Index(['Sector', 'Sub-Sector', 'Org Name', 'Year', 'WC', '1. Sales',
       '1. Total capital employed (C+D)',
       '10. Profit / (loss) after tax (F8-F9)', '11. Cash dividends',
       '12. Bonus shares / stock dividends', 'MVE', 'RE',
       '3. Gross profit / (loss) (F1-F2)',
       '4. General, administrative and other expenses', '4. Intangible assets',
       '4. Salaries, wages and employee benefits', 'TL',
       '6. Contractual liabilities (H5+E2(i))', 'EBIT',
       '6. Other non-current assets', '7. Financial expenses',
       '7. Purchases (F2+Current year B2 - Prev. Year B2)',
       '8. Operating cash flow (F6+H3-F9(i(a))',
       '8. Profit / (loss) before taxation (F6-F7)', '9. Tax expenses',
       'C. Shareholders' Equity (C1+C2+C3)',
       'D. Non-Current Liabilities (D1+D2+D3+D4+D5)',
       'E. Current Liabilities (E1+E2+E3+E4)', 'I. Key Performance Indicators',
       'L2. Quick ratio (B1+B3+B5 

,Sector,Sub-Sector,Org Name,Year,Altman Z-Score
0,Cement,Cement,380001 - Power Cement,2017,37.409469
1,Cement,Cement,380001 - Power Cement,2018,7.737906
2,Cement,Cement,380001 - Power Cement,2019,7.922585
3,Cement,Cement,380001 - Power Cement,2020,6.477189
4,Cement,Cement,380001 - Power Cement,2021,4.196287
...,...,...,...,...,...
2250,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340272 - Amtex Ltd.,2022,7.150851
2251,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340793 - INTERLOOP (Pvt) Ltd.,2019,12.109025
2252,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340793 - INTERLOOP (Pvt) Ltd.,2020,8.020518
2253,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340793 - INTERLOOP (Pvt) Ltd.,2021,9.733500


In [11]:
    # # Initialize and process data
    # processor = StateBankDataExtraction('FSA(work).xlsx')
    # processor.load_excel(sheet_name='FSA NF 2017-22', header=3) \
    #          .handle_duplicates() \
    #          .fill_empty_rows() \
    #          .filter_data() \
    #          .filter_variables()
    #          # .display_data()

    # # Get the processed and reshaped DataFrame
    # processing_df = processor.processing_variable()
    # # print('Processing Variables Dataframe')
    # # display(processing_df)

    # # # Get the processed and reshaped DataFrame
    # # dashboard_df = processor.dashboard_variable()
    # # print('Dashboard Variables Dataframe')
    # # display(dashboard_df)

    # # Initialize AltmanScore with the reshaped DataFrame
    # analyzer = AltmanScore(processing_df)
    # analyzer.calculate_altman_score()
    # # Calculate Altman Z-Scores
    # altman_avg_for_each_org_of_each_year = analyzer.cal_altman_for_each_org_of_each_year(True)
    # altman_avg_for_each_org_avg = analyzer.cal_altman_for_each_org_avg(True)
    # altman_avg_for_each_sub_sector = analyzer.cal_altman_for_each_sub_sector(True)
    # altman_avg_for_each_sector = analyzer.cal_altman_for_each_sector(True)

    # # Display results
    # print('Altman Avg For Each Org Of Each Year')
    # display(altman_avg_for_each_org_of_each_year)
    # print('Altman Avg For Each Org')
    # display(altman_avg_for_each_org_avg)
    # print('Altman Avg For Each Sub Sector')
    # display(altman_avg_for_each_sub_sector)
    # print('Altman Avg For Each Sector')
    # display(altman_avg_for_each_sector)


In [12]:
class FinancialMetrics:
    def __init__(self, df_FM):
        self.df_FM = df_FM

    def cal_independent_variables(self):
        df_FM = self.df_FM.copy()

        # Rename columns
        df_FM.rename(columns={
            'Total Assets (A+B) / Equity & Liabilities (C+D+E)': 'TA',
            '1. Sales': 'S',
            '4. Intangible assets': 'IntA',
            '11. Cash dividends': 'CD',
            'of which: i) Interest / markup payables': 'IM',
            'S2. Debt to assets ratio (  D+E as % of Avg. {Current year(A+B),previous year (A+B)})': 'DA',
            'S1. Debt equity ratio [(D+E) to C]': 'DE'
        }, inplace=True)
        
        # Print to debug
        # print("DataFrame after replacement and renaming:")
        # print(df_FM.head())

        # Calculations
        df_FM['FS'] = np.log(df_FM['TA'])
        df_FM['GO'] = df_FM['S'] / df_FM['TA']
        df_FM['IA'] = df_FM['IntA']
        df_FM['DIVID'] = df_FM['CD']
        df_FM['IP'] = df_FM['IM']
        df_FM['Debt_TA'] = df_FM['DA']
        df_FM['Debt_EQ'] = df_FM['DE']

        # Print to debug
        # print("DataFrame after dropping NaNs:")
        # print(df_FM.head())

        self.df_FM = df_FM
        return df_FM

    def cal_ind_var_for_each_org_of_each_year(self, save=False, filename='cal_ind_var_for_each_org_of_each_year.xlsx'):
        pivotted_df = self.df_FM[['Sector', 'Sub-Sector', 'Org Name', 'Year', 'FS', 'GO', 'IA', 'DIVID', 'IP', 'Debt_TA', 'Debt_EQ']]
        if save:
            pivotted_df.to_excel(filename, index=False)
            print(f"DataFrame saved to '{filename}'")
        
        return pivotted_df

    def cal_ind_var_for_each_org_avg(self, save=False, filename='cal_ind_var_for_each_org_avg.xlsx'):
        df_avg_metrics = self.df_FM.groupby(['Sector', 'Sub-Sector', 'Org Name'])[['FS', 'GO', 'IA', 'DIVID', 'IP', 'Debt_TA', 'Debt_EQ']].mean().reset_index()
        df_avg_metrics.rename(columns={'FS': 'Average FS', 'GO': 'Average GO', 'IA': 'Average IA', 'DIVID': 'Average DIVID', 'IP': 'Average IP', 'Debt_TA': 'Average Debt_TA', 'Debt_EQ': 'Average Debt_EQ'}, inplace=True)
        if save:
            df_avg_metrics.to_excel(filename, index=False)
            print(f"DataFrame saved to '{filename}'")
        
        return df_avg_metrics
        
    def cal_ind_var_for_each_sub_sector(self, save=False, filename='cal_ind_var_for_each_sub_sector.xlsx'):
        df_avg_metrics_subsector = self.df_FM.groupby(['Sector', 'Sub-Sector'])[['FS', 'GO', 'IA', 'DIVID', 'IP', 'Debt_TA', 'Debt_EQ']].mean().reset_index()
        df_avg_metrics_subsector.rename(columns={'FS': 'Average FS', 'GO': 'Average GO', 'IA': 'Average IA', 'DIVID': 'Average DIVID', 'IP': 'Average IP', 'Debt_TA': 'Average Debt_TA', 'Debt_EQ': 'Average Debt_EQ'}, inplace=True)
        if save:
            df_avg_metrics_subsector.to_excel(filename, index=False)
            print(f"DataFrame saved to '{filename}'")
        
        return df_avg_metrics_subsector

    def cal_ind_var_for_each_sector(self, save=False, filename='cal_ind_var_for_each_sector.xlsx'):
        df_avg_metrics_sector = self.df_FM.groupby(['Sector'])[['FS', 'GO', 'IA', 'DIVID', 'IP', 'Debt_TA', 'Debt_EQ']].mean().reset_index()
        df_avg_metrics_sector.rename(columns={'FS': 'Average FS', 'GO': 'Average GO', 'IA': 'Average IA', 'DIVID': 'Average DIVID', 'IP': 'Average IP', 'Debt_TA': 'Average Debt_TA', 'Debt_EQ': 'Average Debt_EQ'}, inplace=True)
        if save:
            df_avg_metrics_sector.to_excel(filename, index=False)
            print(f"DataFrame saved to '{filename}'")
        
        return df_avg_metrics_sector



In [13]:
# Get financial metrics from state bank data
fm = FinancialMetrics(sb_df)
fm.cal_independent_variables()
fm_df = fm.cal_ind_var_for_each_org_of_each_year(True)
display(fm_df)

C:\Users\Guru Group\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


DataFrame saved to 'cal_ind_var_for_each_org_of_each_year.xlsx'


,Sector,Sub-Sector,Org Name,Year,FS,GO,IA,DIVID,IP,Debt_TA,Debt_EQ
0,Cement,Cement,380001 - Power Cement,2017,16.247993,0.393482,8977.0,0.00,11214.0,0.33952,0.35654
1,Cement,Cement,380001 - Power Cement,2018,17.014890,0.177150,1077.0,0.00,249507.0,0.73630,1.16985
2,Cement,Cement,380001 - Power Cement,2019,17.511193,0.095807,0.0,0.00,1153972.0,0.88592,2.47973
3,Cement,Cement,380001 - Power Cement,2020,17.633249,0.090401,5493.0,0.00,1777146.0,0.86326,4.36657
4,Cement,Cement,380001 - Power Cement,2021,17.633040,0.312598,2958.0,0.00,1258520.0,0.76372,3.23379
...,...,...,...,...,...,...,...,...,...,...,...
2250,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340272 - Amtex Ltd.,2022,15.113294,0.341660,0.0,0.00,2735144.0,3.52416,-1.40264
2251,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340793 - INTERLOOP (Pvt) Ltd.,2019,17.519672,0.923554,70083.0,2616592.50,113942.0,0.56874,1.31881
2252,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340793 - INTERLOOP (Pvt) Ltd.,2020,17.627712,0.813198,175813.0,1744395.00,210282.0,0.67512,1.78203
2253,Textile Sector,"Spinning, Weaving, Finishing of Textiles",340793 - INTERLOOP (Pvt) Ltd.,2021,17.921369,0.905551,209623.0,2180493.75,221674.0,0.75851,1.95862


In [14]:
# # Assuming df_financialMetrics is your input DataFrame

# # Instantiate the class with your DataFrame
# metrics_calculator = FinancialMetrics(processing_df)

# # Calculate the independent variables (financial metrics)
# metrics_calculator.cal_independent_variables()

# # Get the metrics for each organization for each year
# df_org_year_metrics = metrics_calculator.cal_ind_var_for_each_org_of_each_year(True)

# # Get the average metrics for each organization
# df_avg_org_metrics = metrics_calculator.cal_ind_var_for_each_org_avg(True)

# # Get the average metrics for each sub-sector
# df_avg_sub_sector_metrics = metrics_calculator.cal_ind_var_for_each_sub_sector(True)

# # Get the average metrics for each sector
# df_avg_sector_metrics = metrics_calculator.cal_ind_var_for_each_sector(True)

# # View the result
# print('df_org_year_metrics')
# display(df_org_year_metrics)
# print('df_avg_org_metrics')
# display(df_avg_org_metrics)
# print('df_avg_sub_sector_metrics')
# display(df_avg_sub_sector_metrics)
# print('df_avg_sector_metrics')
# display(df_avg_sector_metrics)

In [15]:
import pandas as pd

class MergeDatasets:
    def __init__(self, altman_df, fm_df, worldbank_df):
        self.altman_df = altman_df
        self.fm_df = fm_df
        self.worldbank_df = worldbank_df

    def merge_data(self):
        # Ensure 'Year' column is the same type in all datasets (convert to string or int)
        self.altman_df['Year'] = self.altman_df['Year'].astype(int)
        self.fm_df['Year'] = self.fm_df['Year'].astype(int)
        self.worldbank_df['Year'] = self.worldbank_df['Year'].astype(int)
        # Merge dataset1 and dataset2 (altman_df and fm_df)
        merged_df = pd.merge(self.altman_df, self.fm_df, on=['Sector', 'Sub-Sector', 'Org Name', 'Year'], how='outer')
        # Merge with dataset3 (worldbank_df) on 'Year'
        final_df = pd.merge(merged_df, self.worldbank_df, on='Year', how='inner')  # or how='outer'
        return final_df

    def save_to_excel(self, filename='final_df.xlsx'):
        """Save the final merged dataframe to an Excel file."""
        final_df = self.merge_data()
        final_df.to_excel(filename, index=False)
        print(f"Data saved to {filename}")
        return final_df


# df1 = altm_sb_df
# df2 = fm_df
# df3 = wrldb_df

# Create instance of MergeDatasets by passing the DataFrames
merger = MergeDatasets(altm_sb_df, fm_df, wrldb_df)

# Execute merging and save the result to an Excel file
final_df = merger.save_to_excel('final_output.xlsx')


C:\Users\Guru Group\AppData\Local\Temp\ipykernel_13604\483707012.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.altman_df['Year'] = self.altman_df['Year'].astype(int)
C:\Users\Guru Group\AppData\Local\Temp\ipykernel_13604\483707012.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.fm_df['Year'] = self.fm_df['Year'].astype(int)


Data saved to final_output.xlsx


### BELOW IS THE WORKING FOR PREDICTIVE ALGORITHM

In [17]:
# DEALING WITH MISSING AND INFINITE VALUES
# FOR  APPLYING PREDICTIVE ALGORITHM

# Check for NaN and Infinite values only in numeric columns
numeric_cols = df.select_dtypes(include=[np.number])

# Check for NaN and Infinite values in numeric columns
nan_counts_numeric = numeric_cols.isna().sum()
inf_counts_numeric = np.isinf(numeric_cols).sum()

print("NaN Counts:", nan_counts_numeric)
print("Infinite Counts:", inf_counts_numeric)


# Fill missing values with the mean of the respective column
df['Poverty headcount ratio at national poverty lines (% of population)'] = \
    df['Poverty headcount ratio at national poverty lines (% of population)'].fillna(df['Poverty headcount ratio at national poverty lines (% of population)'].mean())

df['Unemployment, total (% of total labor force) (national estimate)'] = \
    df['Unemployment, total (% of total labor force) (national estimate)'].fillna(df['Unemployment, total (% of total labor force) (national estimate)'].mean())


# Ensure all numeric columns are in standard float format and round to 5 decimal places
# List of specific columns you want to convert and round
columns_to_convert = ['Altman Z-Score', 'FS', 'GO', 'IA', 'DIVID', 'IP', 'Debt_TA', 'Debt_EQ', 'Broad money (% of GDP)', 'Broad money growth (annual %)', 'CPIA transparency, accountability, and corruption in the public sector rating (1=low to 6=high)', 'Control of Corruption: Estimate', 'Current account balance (% of GDP)', 'Current account balance (BoP, current US$)', 'GDP (constant LCU)', 'GDP growth (annual %)', 'GDP per capita (constant 2015 US$)', 'GDP per capita (current US$)', 'Government Effectiveness: Estimate', 'Inflation, consumer prices (annual %)', 'Official exchange rate (LCU per US$, period average)',	'Political Stability and Absence of Violence/Terrorism: Estimate', 'Poverty headcount ratio at national poverty lines (% of population)', 'Real effective exchange rate index (2010 = 100)', 'Rule of Law: Estimate', 'Total debt service (% of exports of goods, services and primary income)', 'Unemployment, total (% of total labor force) (national estimate)']  # Replace with your actual column names

# Convert specific columns to numeric (handling scientific notation) and round to 5 decimal places
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce').round(5)


df = df.round(5)  # Round all numeric columns to 5 decimal places

# Alternatively, you can round specific columns only:
df['column_name'] = df['column_name'].round(5)
df.to_excel("filled_output.xlsx")

NameError: name 'df' is not defined

In [ ]:
# Function to create sequences for LSTM input
def create_sequences(data, n_steps):
    sequences = []
    targets = []
    
    for i in range(len(data) - n_steps):
        seq = data[i:i + n_steps]
        target = data[i + n_steps]
        sequences.append(seq)
        targets.append(target)
    
    return np.array(sequences), np.array(targets)

import numpy as np
from sklearn.preprocessing import MinMaxScaler

def preprocess_data_for_lstm(df, feature_columns):
    data_dict = {}
    
    # Group by organization to prepare data for each organization separately
    for org, org_data in df.groupby('Org Name'):
        org_data = org_data.sort_values(by='Year')  # Sort by Year
        num_years = len(org_data)
        
        if num_years < 2:
            # Skip organizations with fewer than 2 years of data
            continue
        
        # Separate features and target
        X = org_data[feature_columns].values
        y = org_data['Altman Z-Score'].values  # Assuming 'Target' is your dependent variable
        
        # Split into training and test sets: all except the last year for training, last year for testing
        X_train, y_train = X[:-1], y[:-1]
        X_test, y_test = X[-1:], y[-1:]
        
        # If training data is empty, skip this organization
        if X_train.shape[0] == 0:
            print(f"Skipping organization {org} due to insufficient training data.")
            continue
        
        # Store in the dictionary
        data_dict[org] = {
            'X_train': X_train,
            'y_train': y_train,
            'X_test': X_test,
            'y_test': y_test
        }
    
    return data_dict

# Define the LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))  # Predicting a single value (Altman Z-Score)
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


def train_and_evaluate_lstm(data_dict):
    results = {}

    for org, data in data_dict.items():
        # Use the correct keys from the data_dict
        X_train, y_train = data['X_train'], data['y_train']
        X_test, y_test = data['X_test'], data['y_test']
        
        # Print shapes for debugging
        print(f"Shape of X_train: {X_train.shape}")
        
        # Build and train the model
        model = build_lstm_model((X_train.shape[1], 1))  # Assuming one target feature
        model.compile(optimizer='adam', loss='mean_squared_error')
        
        # Train the model
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
        
        # Predict on the test set
        predictions = model.predict(X_test)
        
        # Store the predictions for this organization
        results[org] = {
            'predictions': predictions,
            'y_test': y_test
        }
    
    return results

# Columns to use for LSTM model (you can adjust based on your dataset)
feature_columns = ['FS', 'GO', 'IA', 'DIVID', 'IP', 'Debt_TA', 'Debt_EQ', 'Broad money (% of GDP)', 'Broad money growth (annual %)', 'CPIA transparency, accountability, and corruption in the public sector rating (1=low to 6=high)', 'Control of Corruption: Estimate', 'Current account balance (% of GDP)', 'Current account balance (BoP, current US$)', 'GDP (constant LCU)', 'GDP growth (annual %)', 'GDP per capita (constant 2015 US$)', 'GDP per capita (current US$)', 'Government Effectiveness: Estimate', 'Inflation, consumer prices (annual %)', 'Official exchange rate (LCU per US$, period average)',	'Political Stability and Absence of Violence/Terrorism: Estimate', 'Poverty headcount ratio at national poverty lines (% of population)', 'Real effective exchange rate index (2010 = 100)', 'Rule of Law: Estimate', 'Total debt service (% of exports of goods, services and primary income)', 'Unemployment, total (% of total labor force) (national estimate)']  # Replace with actual financial indicators

# Preprocess the data
data_dict = preprocess_data_for_lstm(df, feature_columns)

# Train and evaluate the LSTM model
results = train_and_evaluate_lstm(data_dict)

# Loop through the results for each organization
for org_name, result in results.items():
    predicted_value = result['predictions'][0][0]  # Extracting the predicted value
    y_test_value = result['y_test'][0]  # Actual value
    print(f"Organization: {org_name}\n Predicted Value: {predicted_value}\n Actual Value: {y_test_value}\n")
